# Extração da Drogaria Minas Mais

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
import json
import pandas as pd

In [ ]:
# Capturando todas as URLs

# Função de acessar site
all_workers = os.cpu_count()

def access_site(url, max_attempts=all_workers):
    headers = {'Accept': '*/*', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for retry in range(max_attempts):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
        except Exception as e:
                print(f"An error occurred while accessing URL: {url}: {e}. Retrying... (Attempt {retry + 1}/{max_attempts})")
                time.sleep(1)

# Função captura de todos os nomes de marcas e criação de URLs
def getNomeMarcas():
    response = access_site("https://www.drogariasminasmais.com.br/medicamentos")
    soup = BeautifulSoup(response.text, 'html.parser')
    sessaoMarcasGrandes = soup.find('div', class_='vtex-search-result-3-x-filter__container vtex-search-result-3-x-filter__container--responsive-result-content-mz0001 bb b--muted-4 vtex-search-result-3-x-filter__container--brand')
    sessaoMarcas = sessaoMarcasGrandes.find('div', class_='vtex-search-result-3-x-filterTemplateOverflow vtex-search-result-3-x-filterTemplateOverflow--responsive-result-content-mz0001 pb5 overflow-y-auto')
    marcas = sessaoMarcas.find_all('label', class_='vtex-checkbox__label w-100 c-on-base pointer')
    return marcas

# Criando uma lista de URLs ainda não bem estruturadas
urlSuja = getNomeMarcas()
urlLimpa = []

# Limpando lista das URLs
for i in urlSuja:
    # Criando texto separado e minusculo com "-"
    texto = i.text.lower()
    textoNovo = texto.replace(" ", "-")

    # Salvando tudo em uma lista
    urlLimpa.append(textoNovo)

listaUrls = []

# Criando todas as URLs
for j in urlLimpa:
    URL = "https://www.drogariasminasmais.com.br/medicamentos/" + j + "?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page="
    listaUrls.append(URL)

In [ ]:
# Mudando o número das páginas

# Função para encontrar um arquivo JSON secreto dependendo da página e da marca
def getJSON(urlInicial, page_number):
    url = str(urlInicial) + str(page_number)
    requisicao = requests.get(url)
    soup = BeautifulSoup(requisicao.text, 'html.parser')
    string = soup.find_all('script')[-34].text
    return string

jsonFinal = getJSON("https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=", 1)

In [ ]:
# Criando o JSON e capturando todos os códigos de produtos
jsonFiltrado = jsonFinal[3545215:-8]
dados = json.loads(jsonFiltrado)
listaProdutos = []
listaProdutosCerta = []
listaProdutosSuperCerta = []
j = 0 

for i in dados.keys():
    if j % 18 == 0:
        listaProdutos.append(i)
    j = j + 1
    if j == 270:
        break

for i in listaProdutos:
    listaProdutosCerta.append(i.split(".")[0])

for i in listaProdutosCerta:
    listaProdutosSuperCerta.append(i.replace("$", ""))

In [ ]:
# Pegando todos os dados dos produtos
ean = []
sku = []
precoComDesconto = []
precoSemDesconto = []
desconto = []
marcas = []

for i in listaProdutosSuperCerta:
    marcas.append(dados[i]["brand"])

for i in listaProdutosSuperCerta:
    precoComDesconto.append(dados[i][])
    precoSemDesconto.append(dados[i][])
    desconto.append(dados[i][])

for i in listaProdutosSuperCerta:
    ean.append(dados[i][])
    sku.append(dados[i][])

In [ ]:
# Capturando todos os dados dentro do JSON

# Função para encontrar um arquivo JSON secreto dependendo da página e da marca
def getJSON(urlInicial, page_number):
    url = str(urlInicial) + str(page_number)
    requisicao = requests.get(url)
    soup = BeautifulSoup(requisicao.text, 'html.parser')
    string = soup.find_all('script')[-34].text
    print(string)
    return string

total_pages = 25
listaJSON = []

# Iteração por páginas de usuários
for i in listaUrls: 
    for page_number in range(1, total_pages + 1):
        listaJSON.append(getJSON(i, page_number))